# CNN for Medical Image Segmentation

**Welcome to the CNN tutorial for Chest X-Ray Image Segmentation!**
Medical image segmentation and medical image classification are very different tasks. Classification simply aims to classify images into a number of predefined groups, or classes. For example, the Medical Image Classification tutorial was made to classify chest x-ray images into those which displayed normal pathology or signs of pneumonia.<br/>
<br/>
Medical image segmentation, on the other hand aims to look at images and segment them into components, using a mask. Specifically, this tutorial will cover semantic segmentation in which we will try to train a fully convolutional neural network to create image masks from chest x-rays that cover the area of the image which shows the lungs; segmenting the image into lung tissue and non-lung tissue.<br/>
<br/>
The neural network being used here is based off work by *Long et al* in their paper, <a href="https://arxiv.org/abs/1411.4038">*Fully Convolutional Networks for Semantic Segmentation*</a>. We should find that the network will produce somewhat blurry and not very accurate masks. This is somewhat a result of the dataset being used but also of the properties of this neural network's architecture. A more accurate and cutting edge model will be investigated in the UNet and k-Dense UNet tutorials, but it is recommended that you try this one out first!<br/>
<br/>
If you've not tried the CNN for Medical Image *Classification* tutorial, complete that one first as in contains a more introductory approach to CNNs. Furthermore, this tutorial uses data from Kaggle to segment lung tissue in chest x-rays. You can get the dataset <a href="https://www.kaggle.com/nikhilpandey360/chest-xray-masks-and-labels">HERE</a>.

**You will need to create a verified Kaggle account, download the data and change the source directory in this file.**

Let's get started!

## Importing Packages and Dependencies
To get started, we need to import the required packages which the code will use. The architecture of the neural network will use Keras. 

Keras is a high level machine learning API which uses TensorFlow as a backend. This allows users to focus on implementation from a more blackbox perspective. TensorFlow is much better suited for more whitebox approaches. TensorFlow uses a structure called tensors which a basically n-dimensional arrays or matricies. We can use NumPy arrays to pass to TensorFlow and Keras.

In [ ]:
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
from skimage.io import imread, imshow
from skimage.transform import resize

img_dims = 256 # should be atleast 256px
my_epochs = 20
batch_size = 32

## Importing Images
We now need to set an input path from which to load images. **You must change the following line to point to the correct folder on your system.**

In [ ]:
inPath = "C:\Datasets\LungSegmentation\Lung Segmentation"

## Testing the Filepath
You can verify that the path has been set correctly by counting the images from the dataset.

The for loop below simply counts and displays the number of images in each directory. This gives us an idea of how many images we are dealing with in each set. You should confirm that your output is as follows:<br />
`Set: /CXR_png
Images: 800
Set: /masks
Images: 704
Set: /test
Images: 96`

In [ ]:
train = len(os.listdir(inPath + "/CXR_png"))
masks = len(os.listdir(inPath + "/masks"))
tests = len(os.listdir(inPath + "/test"))

print("Set: " + "/CXR_png" + "\nImages: " + str(train) + "\nSet: " + "/masks" + "\nImages: " + str(masks) + "\nSet: " + "/test" + "\nImages: " + str(tests))

## Preparing the Data
Confirming that the we have our dataset accessed appropriately, we also need to assess how to properly prepare this data for the model. Upon inspection, it seems like 704 masks are generated from 704 of the original images in CXR_png. By examining the image names, we also find that no masks are created for the 96 test cases. This means we'll need to either (a) organize the images into folders which are more easily accessed or (b) process the data at runtime without reorganizing files. <br/>
<br/>
Also, upon further inspection, the dimensions of the images being used range from 1130x948 px to nearly 5000x5000 px. This is an important consideration for the determining what the dimensions of input layer of the CNN should be. We should also consider that the masks range in size from 1255x989 px to 4892x4020 px. This needs to be considered in order to determine what size the output layer should be.<br/>
<br/>
Finally, because of the last point, we make the assumption that resizeing that images will not affect the accuracy of the training (heads up: it will).


Having the test images repeated in the /CXR_png and /test directories is redundant, so to make things easier, I'm going to clean up /CXR_png by removing all the images that are present in the /test folder. This leaves only training images in the /CXR_png directory which can then be used to gather training images without having to check each image for having a corresponding mask.<br/>
<br/>
I'll do that with this code:

In [ ]:
for entry in os.scandir(inPath + "/test"):
    if entry.is_file():
        if Path(inPath + "/CXR_png/" + entry.name):
            # print(entry.name)
            os.remove(inPath + "/CXR_png/" + entry.name)

            
# Confirm that the command worked as expected
train = len(os.listdir(inPath + "/CXR_png"))
masks = len(os.listdir(inPath + "/masks"))
tests = len(os.listdir(inPath + "/test"))

print("Set: " + "/CXR_png" + "\nImages: " + str(train) + "\nSet: " + "/masks" + "\nImages: " + str(masks) + "\nSet: " + "/test" + "\nImages: " + str(tests))

With the directories cleaned up, we can now prepare the data. We'll do this a little differently that the CNN for classification tutorial. This time we will create two matrices of zeros which was have their contents filled with the images. One of these matrices will contain all of the training images and the other will contain all of the masks. We read in the data from the files to the new matrices.<br/>
<br/>
Note that we import a package called tqdm which just provides a progress bar to display while the loop is iterating.

In [ ]:
from tqdm import tqdm

train_imgs = os.listdir(inPath + "/CXR_png")
mask_imgs = os.listdir(inPath + "/masks")
test_imgs = os.listdir(inPath + "/test")

X_train = np.zeros((len(train_imgs), img_dims, img_dims, 3), dtype=np.uint8)
Y_train = np.zeros((len(mask_imgs), img_dims, img_dims, 1), dtype=np.uint8)

for n, file in tqdm(enumerate(train_imgs), total=len(train_imgs)):
    t_img = imread(inPath + "/CXR_png/" + file)
    t_img = resize(t_img, (img_dims, img_dims, 3), mode='constant', preserve_range=True)
    X_train[n] = t_img
    if Path(inPath + "/masks/" + file).exists():
        m_mask = imread(inPath + "/masks/" + file)
    else:
        m_mask = imread(inPath + "/masks/" + file[:-4] + "_mask.png")
    m_mask = resize(m_mask, (img_dims, img_dims, 1), mode='constant', preserve_range=True)
    Y_train[n] = m_mask

We also need to prepare the test data. This can be done in a similar manner.

In [ ]:
X_test = np.zeros((len(train_imgs), img_dims, img_dims, 3), dtype=np.uint8)

for n, file in tqdm(enumerate(test_imgs), total=len(test_imgs)):
    test_img = imread(inPath + "/test/" + file)
    test_img = resize(test_img, (img_dims, img_dims, 3), mode='constant', preserve_range=True)
    X_test[n] = test_img

## Defining the CNN Architecture
The most important step! Keras makes creating neural networks extremely intuitive and easy. In fact, it may seem like all the previous steps are the most difficult to understand initially. <br />
To generate this CNN, we must first contruct each layer in the neural network. We'll walk through Abhinav's CNN layer by layer.<br/>
But, before we get started, we need to create an input tensor (a matrix) that is the dimensions of the images being used to train the neural network and an added third dimension of 3. This third dimension is due to the three channels used to compose JPEG images (height x width x channel) which are actually three dimensional array data structures!

In [ ]:
inputs = tf.keras.layers.Input((img_dims, img_dims, 3))

Now we create the first layers in the CNN, which takes `inputs`, the JPEG images, as an input. The convolutional layer scans a kernel across the image and produces another tensor which maps the activation of the kernel across the image. This is the first step in feature extraction. The resulting tensor created from this convolutional layer is then passed to another convolutional layer, and the tensor output by this layer is passed to a maxpooling layer that reduces dimensionality of the data and helps the network to learn and extract features from the image produced the convolution. This completes the first convolutional block.

In [ ]:
#first convolutional block
c1 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same')(inputs)
c1 = tf.keras.layers.Dropout(rate=0.2)(c1)
c1 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same')(c1)
c1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c1)

#second convolutional block
c2 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same')(c1)
c2 = tf.keras.layers.Dropout(rate=0.2)(c2)
c2 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same')(c2)
c2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c2)

#third convolutional block
c3 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same')(c2)
c2 = tf.keras.layers.Dropout(rate=0.2)(c3)
c3 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same')(c3)
c3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c3)

#Upsampling and fourth convolutional block
u1 = tf.keras.layers.UpSampling2D(size=(2,2))(c3) 
u1 = tf.keras.layers.concatenate([u1, c2], axis=-1)
c4 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same')(u1)
c4 = tf.keras.layers.Dropout(rate=0.2)(c4)
c4 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same')(c4)

#Upsampling and fifth convolutional block
u2 = tf.keras.layers.UpSampling2D(size=(2,2))(c4) 
u2 = tf.keras.layers.concatenate([u2, c1], axis=-1)
c5 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same')(u2)
c5 = tf.keras.layers.Dropout(rate=0.2)(c5)
c5 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same')(c5)

Finally, the output layer.

In [ ]:
# Output layer
outputs = tf.keras.layers.Conv2D(filters=1, kernel_size(1,1), activation='sigmoid')(c5)

## Compiling The Model
Now, we can compile the model. But before we do that, there are some things to consider.<br/>
1. Training can take a very long time, from hours possibly to days <br/>
2. If training is interupted, then progress is lost <br/>
3. We can't know the what the perfect number of epochs is that is high enough to prevent underfitting, but low enough to prevent overfitting 


In order to prevent losing a model during training, we can implement the `ModelCheckpoint` callback. This call back saves the model after every epoch to a file at specified `filepath`. If the `save_best_only` boolean is set to `True` then the model will save only the best epochs. The `save_weights_only` boolean means that instead of saving all parameters in the model, just the weights of each connection are saved. <br/>
<br/>
Choosing hyperparameters like epoch, can be done using trial and error, but this is tedious and may cause overfitting if the model is allowed to train for too many epochs. In order to prevent overfitting, we can used both the `ReduceLROnPlateau` and `EarlyStopping` callbacks. The the `ReduceLROnPLateau` callback method reduces the learning rate of the model once learning starts to plateau. The `EarlyStopping` callback stops the training process when a certain metric that is being monitored stops improving.<br/><br/>
For `ReduceLROnPlateau`, the value which we'd like to monitor is the model's loss, so `monitor` is set to `val_loss`. The factor value is factor by which the learning rate will be reduced. `patience` is the number of consecutive epochs with no improvements to wait before reducing learning rate, while `verbose` simply provides a warning message saying that the learning rate is being reduced. Finally, `mode` tells the method to look for improvements in increasing loss when set to `'max'`. When set to `'min'`, the method looks for decreasing loss. In total, this method is saying, monitor`val_loss` as it approaches some `max`imum. If the loss fails to increase for `2` consecutive epochs, reduce the learning rate by a factor of `0.3` and warn the user.<br/><br/>
For `EarlyStopping`, the value being monitor is again `val_loss`, just as in the previous method. There is also another parameter, `min_delta` which is set to `0.1`. This parameter specifies the minimum change in the monitored value to be concidered an improvement. `mode` is set to `min` this time because we want the method to detect when `val_loss` reduces. Finally, the `patience` parameter is set to 3 so as to not interfere with `ReduceLSOnPlateau`.
<br/>
<br/>
To read more about the <a href="https://keras.io/api/callbacks/reduce_lr_on_plateau/">`ReduceLROnPlateau`</a> and <a href="https://keras.io/api/callbacks/early_stopping/">`EarlyStopping`</a> methods (actually they are classes), then see the appropriate documentation by clicking them.


In [ ]:
# Creating model and compiling
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Callbacks
checkpoint = ModelCheckpoint(filepath='xray_model.hdf5', save_best_only=True, save_weights_only=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=2, mode='max')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.1, patience=3, mode='min')
board = TensorBoard(log_dir='logs')

my_callbacks = [
    checkpoint,
    #lr_reduce,
    early_stop,
    board
]

## Model Training
Finally, we can train the model! 
<br/>
The `model.fit()` method trains the model and stores training history to help visualize and track the training process. 
<br/>
We're using a few parameters here with `model.fit()`. The first, `train_gen`, passes the training set generator which was definined above with the line: <br/>
`train_gen, test_gen, test_data, test_labels = process_data(img_dims, batch_size)`.<br/> 
Next, the `epochs` parameter sets the number of training runs, epochs, to perform training on. We defined `my_epochs` above to be 100. The `callbacks` parameter is used to set the callbacks we want the program to use during training. These were diffined above as `my_callbacks`. 

In [ ]:
hist = model.fit(
            X_train,
            Y_train,
            validation_split=0.1,
            epochs=my_epochs,
            callbacks=my_callbacks
)

Using the history, we can create loss and accuracy plots to visualize model training.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 3))
ax = ax.ravel()

for i, met in enumerate(['accuracy', 'loss']):
    ax[i].plot(hist.history[met])
    ax[i].plot(hist.history['val_' + met])
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].legend(['train', 'val'])

We can also gather more useful data like accuracy and pricision by calculating a confusion matrix and other test metrics. Abhinav uses the following code.

## Saving Models
So you've trained the model, great. But we want to actually use the model that we've trained, otherwise, what good is it?
<br/>
Luckily, TensorFlow and Keras make it very easy to save a complete model for further use and/or further training later.
<br/>
This can be done very easily.

In [ ]:
## model.save('path/to/file/filename')
model.save('CNN_Segementation_example.h5')

## New Prediction
Now, we'll try a prediction.

In [ ]:
## Normal X-ray should score closely to 0
img_path = 'CHNCXR_0025_0.png'
img = image.load_img(img_path, target_size=(144, 144))
pred = image.img_to_array(img)
pred = np.expand_dims(pred, axis=0)
pred = pred.astype('float32')/255

prediction = model.predict(pred)

plt.imshow(prediction[0])


## DONE!
Now that you've completed the tutorial, you should go through and experiment with different hyperparameters. <br/><br/>
I would recommend considering the following:
1. Should both `ReduceLROnPlateau` and `EarlyStopping` be used?
2. If both are used, what parameters should be changed to ensure they don't interfere with eachother and still allow the model to train? (hit, you should have at least 10-15 epochs for this particular model with this data to be trained effectively.
3. How does adjusting batch size affect learning and the speed of learning?
4. What structural changes can be made to the CNN that could make it perform better? (I would definitely exeperiment here with different CNN architectures!)
5. After you've experimented with this data, try to modify to the code to use another dataset of your choice. Importantly, if you do so, you must ensure that the images are atleast 150x150 px in size.
6. Try changing the CNN to take a larger image size. Does is take longer to train? Is it more accurate?